In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import plotly
import plotly.express as px
import plotly.graph_objects as go
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# path="/kaggle/input/amazon-top-50-bestselling-books-2009-2019/"

## Step 1: Understand the data

In [ ]:
data=pd.read_csv("/kaggle/input/amazon-top-50-bestselling-books-2009-2019/bestsellers with categories.csv")

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.select_dtypes(include='object').columns

In [ ]:
data.select_dtypes(exclude ='object').columns

## Step 2: Data Cleaning

### Checking null values

In [ ]:
data.isnull().sum()

#### conclusion: There is no missing value in all the columns

### Checking duplicate values

In [ ]:
data[data.duplicated()]

#### conclusion: There is no duplicate value in all the columns

### Converting all columns to title case so duplicates can be identify easily

In [ ]:
for col in data.select_dtypes(include='object').columns:
    data[col].str.title().str.strip()

In [ ]:
for col in data.select_dtypes(include='object').columns:
    if data[col].duplicated().any():
        print(f"Column {col} contains duplicate records.")
    else:
        print("There is no duplicates.")

#### conclusion: 
        These 3 columns contains duplicate records.
        The Genre column is categorical column so we can't consider that.
        The Author column can contain duplicate because one author can have different books.
        The Name column can't have duplicate because it's a unique name.


In [ ]:
duplicated=data[data.duplicated(['Name'], keep=False)]
duplicated.sort_values('Name')

#### conclusion: The reason of mulitple duplicate is the year column. Let's remove the column for the analysis

In [ ]:
data.drop(['Year'],axis=1,inplace=True)

In [ ]:
data=data.drop_duplicates(keep='first')

In [ ]:
data.info()

In [ ]:
data=data.drop_duplicates(subset='Name',keep='last')

In [ ]:
data

## Step 3: Data Visualization

### This visualization will consider top 10 books 

In [ ]:
grped_1=data.groupby(['Name'])['User Rating'].mean().reset_index()
top_10_Name_UserRating=grped_1.sort_values('User Rating',ascending=False).head(10)

### This visualization will consider top 10 authors 

In [ ]:
grped_2=data.groupby(['Author'])['Name'].sum().reset_index()
top_10_Names_Authors=grped_2.sort_values('Name',ascending=False).head(10)

### This visualization will consider top 10 authors and their User rating

In [ ]:
grped_3=data.groupby(['Author'])['Reviews'].mean().reset_index()
top_10_Authors_Reviews=grped_3.sort_values('Reviews',ascending=False).head(10)

### This visualization will consider top 10 reviews 

In [ ]:
grped_4=data.groupby(['Name'])['Reviews'].sum().reset_index()
top_10_Names_Reviews=grped_4.sort_values('Reviews',ascending=False).head(10)

### This visualization will consider books by genre 

In [ ]:
top_50_book_genre=data.groupby(['Genre'])['Name'].count().reset_index()
top_50_book_genre.sort_values('Name',ascending=False)

In [ ]:
fig = px.scatter(data, x="Reviews", y="Price",color='Genre', hover_data={'Name': True})
fig.update_layout(title='Books Reviews and their price')
fig.show()

In [ ]:
fig = px.scatter(data, x="Reviews", y="User Rating",color='Genre', hover_data={'Name': True})
fig.update_layout(title='Books Reviews and their User Ratings')
fig.show()

In [ ]:
fig = px.scatter(data.sort_values('Reviews',ascending=False).head(25), x="Author", y="Reviews",color='Genre',hover_data={'Author': True})
fig.update_layout(title='Books Authors and their Reviews')
fig.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
plt.hist(data["User Rating"])
plt.title("User Rating Histogram")

plt.show()


In [ ]:
fig = plt.figure(figsize=(10,6))
plt.hist(data["Price"])
plt.title("Price Histogram")
plt.show()

In [ ]:
fig = px.scatter(data.sort_values('Reviews',ascending=False).head(25), x="Reviews", y="Name",color='User Rating', hover_data={'Name': True})
fig.update_layout(title='Top 25 books and their reviews')
fig.show()

In [ ]:
fig = px.violin(data, y="Price", color="Genre", title='Violin of Price')
fig.show()

In [ ]:
def biVariateAnalysis(col1,col2,data):
    plt.figure(figsize=(15,7))
    ax=sns.countplot(x=col1, data=data,hue=col2)
    plt.title('Distribution of '+'%s' %col1+' vs '+'%s' %col2+'.', fontsize=14)
    plt.xlabel(col1)
    plt.xticks(rotation=90)
    plt.ylabel(col2)
    for p in ax.patches:
        ax.annotate(f'{(p.get_height()/len(data)):.2%}', (p.get_x()+0.1, p.get_height()+50),rotation=90)

In [ ]:
biVariateAnalysis('User Rating','Genre',data)

## Conclusion

### Following are some insights on the amazon books 
    1. All books are categorized into 2 parts Fictional and Fictional.Total 191 books are there in Non Fiction and 160 in Fiction.
    2. The price of books are really impacting the review of the books ,the graph explains that the books price in the range of (0-40) recieved more number of review.
    3. The high number of reviews are not helping in getting good user reviews the maximum highest user ratings have review lies in between 0-30K range.
    4. highest number of books are there in the user rating of 4.7.
    5. Maximum books are having price  below 20.
 ### Following are some conclusions on the amazon books 
    The books publication house can make books in the price range of (0-20), that wil help them in increase there sales and increase the popularity of the books.
    The Fiction books are popular due to the less price and more reviews same thing can happen to Non-Fiction books also.
    